In [ ]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import numpy as np
import pandas as pd
import PreClf, MultiPreClf
from sklearn.datasets import fetch_openml
from sklearn.datasets import make_moons

import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# MLP from sklearn
from sklearn.neural_network import MLPClassifier

# MNIST 
X_, y_ = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas")
# Preprocessing normalisation 
X_ = X_ / 255

y_ = y_.astype(int)

# On-hot encoding
encoder = OneHotEncoder(sparse=False)
y_one_hot = encoder.fit_transform(y_.reshape(-1, 1))


X_train, X_test, y_train, y_test = train_test_split(X_, y_one_hot, test_size=0.2, random_state=42)
y_train_index = np.argmax(y_train, axis=1)


# X_train, X_test, y_train, y_test = train_test_split(X_, y_one_hot, test_size=0.2, random_state=42)



c:\Users\dmamed01\OneDrive - Donaldson Company, Inc\Desktop\proj\personal\ml\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# MLP

In [8]:
class MLP(object):
    def __init__(self, X, Y, X_val, Y_val, layers, n_nodes):
        self.X = X
        self.Y = Y
        self.X_val = X_val
        self.Y_val = Y_val

        self.layers = layers
        self.n_nodes = n_nodes

        self._init_weights()

    def _init_weights(self):
        pass

In [9]:
X_train.shape

(56000, 784)

# MLP (torch)

In [5]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [609]:
# Turn data into torch tensors
x_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_index, dtype=torch.long)

In [611]:
train_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)

In [612]:
class MLP_DR(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size):
        super(MLP_DR, self).__init__()
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size

        # layer 1
        self.fc1 = nn.Linear(input_size, hidden_layers[0])

        # layer 2
        self.fc2 = nn.Linear(hidden_layers[0], hidden_layers[1])

        # layer 3
        self.fc3 = nn.Linear(hidden_layers[1], output_size)

    def forward(self, x):

        linear_1 = self.fc1(x)
        act_1 = F.relu(linear_1)

        linear_2 = self.fc2(act_1)
        act_2 = F.relu(linear_2)

        linear_3 = self.fc3(act_2)
        act_3 = F.softmax(linear_3, dim=1)

        return act_3




In [616]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP_DR(784, [20, 10], 10).to(device)

In [614]:
criterion = nn.CrossEntropyLoss()

In [615]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [617]:
num_epochs = 5  # Number of epochs can be adjusted

for epoch in range(num_epochs):
    model.train()
    for data, targets in train_loader:
        data, targets = data.to(device), targets.to(device)  # Move data to GPU

        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/5], Loss: 2.3043
Epoch [2/5], Loss: 2.3011
Epoch [3/5], Loss: 2.3036
Epoch [4/5], Loss: 2.3028
Epoch [5/5], Loss: 2.3038
